## 1. Import Libraries

In [4]:
#!git clone https://github.com/tayfunai/CNN-BASED-CLASSIFICATION-MODELS.git

In [5]:
import os
import time
import random

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image


In [6]:
"""In order force cuDNN to onyl use deterministic
algorithms which might be useful to produce
reproducible results we need following code"""

if torch.cuda.is_available():
  torch.backends.cudnn.deterministic = True

## 2. Model settings

In [7]:
# Setting a random seed
def set_all_seeds(seed):
  os.environ["PL_GLOBAL_SEED"] = str(seed)
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)

In [8]:
# Setting cuDNN and PyTorch algorithmic behavior to deterministic
def set_deterministic():
  if torch.cuda_is_available():
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
  torch.set_deterministic(True)

In [9]:
#### SETTINGS ####

# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.0001
BATCH_SIZE = 256
NUM_EPOCHS = 40

# Architecture
NUM_CLASSES=10

#Other
DEVICE = "cuda" if torch.cuda.is_available else "cpu"

set_all_seeds(RANDOM_SEED)

In [10]:
# Deterministic behavior not yet supported by AdaptiveAvgPool2d
#set_deterministic()


## 3. Import utility functions

In [11]:
import sys

# In order to import modules from a directory one level up.

sys.path.append('/content/CNN-BASED-CLASSIFICATION-MODELS/helper_functions')
sys.path.insert(0, "..")


In [12]:
from helper_evaluate import compute_accuracy, compute_epoch_loss

from helper_data import *

from helper_train import *



## 4. Dataset

In [13]:
!pip install tqdm

In [14]:
## Set random seed##
set_all_seeds(RANDOM_SEED)

### Dataset ###

train_transforms = transforms.Compose([
    transforms.Resize((70,70)),
    transforms.RandomCrop((64, 64)),
    transforms.ToTensor()
])

test_transforms = transforms.Compose([
    transforms.Resize((70,70)),
    transforms.CenterCrop((64, 64)),
    transforms.ToTensor()
])

train_loader, valid_loader, test_loader = get_dataloaders_cifar10(
    batch_size=BATCH_SIZE,
    num_workers=2,
    train_transforms=train_transforms,
    test_transforms=test_transforms,
    validation_fraction=0.1
)

In [15]:
# Checking the dataset
print(f"Training set:\n")
for images, labels in train_loader:
  print(f"Image batch demensions: {images.size()}")
  print(f"Image label dimensions: {labels.size()}")
  print(labels[:10])
  break

# Checking the validation set
print(f"\nValidation set:")
for images, labels in valid_loader:
  print(f"Image batch demensions: {images.size()}")
  print(f"Image label dimensions: {labels.size()}")
  print(labels[:10])
  break

# Checking the testing set
print(f"\nTesting set:")
for images, labels in test_loader:
  print(f"Image batch demensions: {images.size()}")
  print(f"Image label dimensions: {labels.size()}")
  print(labels[:10])
  break

Training set:

Image batch demensions: torch.Size([256, 3, 64, 64])
Image label dimensions: torch.Size([256])
tensor([0, 2, 3, 5, 4, 8, 9, 6, 9, 7])

Validation set:
Image batch demensions: torch.Size([256, 3, 64, 64])
Image label dimensions: torch.Size([256])
tensor([6, 9, 3, 5, 7, 3, 4, 1, 8, 0])

Testing set:
Image batch demensions: torch.Size([256, 3, 64, 64])
Image label dimensions: torch.Size([256])
tensor([3, 8, 8, 0, 6, 6, 1, 6, 3, 1])


## 5. Model

In [16]:
### MODEL ###

class AlexNet(nn.Module):
  def __init__(self, num_classes):
    super(AlexNet, self).__init__()
    self.features = nn.Sequential(
      nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=3, stride=2),

      nn.Conv2d(64, 192, kernel_size=5, padding=2, groups=2),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=3, stride=2),

      nn.Conv2d(192, 384, kernel_size=3, padding=1, groups=2),
      nn.ReLU(inplace=True),

      nn.Conv2d(384, 256, kernel_size=3, padding=1, groups=2),
      nn.ReLU(inplace=True),

      nn.Conv2d(256, 256, kernel_size=3, padding=1, groups=2),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=3, stride=2),
    )
    self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
    self.classifier = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(256*6*6, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace=True),
        nn.Linear(4096, num_classes),
    )

  def forward(self, x):
    x = self.features(x)
    x = self.avgpool(x)
    x = x.view(x.size(0), 256*6*6)
    logits = self.classifier(x)
    probas = F.softmax(logits, dim=1)
    return logits

In [17]:
torch.manual_seed(RANDOM_SEED)
model = AlexNet(NUM_CLASSES)
model = model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

## 6. Model training

In [ ]:
log_dict = train_classifier_simple_v1(num_epochs=NUM_EPOCHS,
                                      model=model,
                                      optimizer=optimizer,
                                      device=DEVICE,
                                      train_loader=train_loader,
                                      valid_loader=valid_loader,
                                      logging_interval=50)

Epoch: 001/040 | Batch 0000/0175 | Loss: 2.3021
Epoch: 001/040 | Batch 0050/0175 | Loss: 2.0390
Epoch: 001/040 | Batch 0100/0175 | Loss: 1.8914
Epoch: 001/040 | Batch 0150/0175 | Loss: 1.8856
***Epoch: 001/040 | Train. Acc.: 32.917% | Loss: 1.734
***Epoch: 001/040 | Valid. Acc.: 33.540% | Loss: 1.707
Time elapsed: 0.96 min
Epoch: 002/040 | Batch 0000/0175 | Loss: 1.6837
Epoch: 002/040 | Batch 0050/0175 | Loss: 1.6665
Epoch: 002/040 | Batch 0100/0175 | Loss: 1.6248
Epoch: 002/040 | Batch 0150/0175 | Loss: 1.5204
***Epoch: 002/040 | Train. Acc.: 40.850% | Loss: 1.572
***Epoch: 002/040 | Valid. Acc.: 41.540% | Loss: 1.556
Time elapsed: 1.85 min
Epoch: 003/040 | Batch 0000/0175 | Loss: 1.5176
Epoch: 003/040 | Batch 0050/0175 | Loss: 1.6019
Epoch: 003/040 | Batch 0100/0175 | Loss: 1.4632
Epoch: 003/040 | Batch 0150/0175 | Loss: 1.2862
***Epoch: 003/040 | Train. Acc.: 48.143% | Loss: 1.388
***Epoch: 003/040 | Valid. Acc.: 48.320% | Loss: 1.385
Time elapsed: 2.73 min
Epoch: 004/040 | Batch 00

## 7. Model Evaluation

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
loss_list = log_dict['train_loss_per_batch']

plt.plot(loss_list, label='Minibatch loss')
plt.plot(np.convolve(loss_list,
                     np.ones(200,)/200, mode='valid'),
         label='Running average')

plt.ylabel('Cross Entropy')
plt.xlabel('Iteration')
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(1, NUM_EPOCHS+1), log_dict['train_acc_per_epoch'], label='Training')
plt.plot(np.arange(1, NUM_EPOCHS+1), log_dict['valid_acc_per_epoch'], label='Validation')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
with torch.set_grad_enabled(False):

    train_acc = compute_accuracy(model=model,
                                 data_loader=test_loader,
                                 device=DEVICE)

    test_acc = compute_accuracy(model=model,
                                data_loader=test_loader,
                                device=DEVICE)

    valid_acc = compute_accuracy(model=model,
                                 data_loader=valid_loader,
                                 device=DEVICE)


print(f'Train ACC: {valid_acc:.2f}%')
print(f'Validation ACC: {valid_acc:.2f}%')
print(f'Test ACC: {test_acc:.2f}%')
